In [2]:
import pandas as pd 
import numpy as np 

In [13]:
data = pd.read_csv('../data/flights_train.csv')

In [7]:
data.head()

,flight_date,from,to,avg_weeks,target,std_weeks
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [19]:
# extract all (from , to) pairs from data as a list 
location_pairs = data.groupby(['from','to']).count().index.tolist()
len(location_pairs)


126

In [26]:
def extract_flight_feat(flight_loc):
    target,source = flight_loc
    flight_ts = data[(data['from']==target) & (data['to']==source)]['target']
    mean = flight_ts.mean()
    std = flight_ts.std()
    return mean , std
flights_stats = { flight_edge : extract_flight_feat(flight_edge) for flight_edge in location_pairs }

In [77]:
class baseline_model():
    def __init__(self):
        self.flights_stats = flights_stats
    
    def predict(self,flight_loc):
        target,source = flight_loc
        mean,std = self.flights_stats[(target,source)]
        return mean #+ np.random.normal(-0.5, 1) * std

base_model = baseline_model()


In [78]:
X_data = data[['from','to']].values
y_data = data['target'].values

y_pred = [base_model.predict(x) for x in X_data]
# score rmse 
from sklearn.metrics import mean_squared_error
score = mean_squared_error(y_data,y_pred)**0.5
score


0.7243219278020451

In [79]:
data_test = pd.read_csv('../data/flights_Xtest.csv')

In [84]:
X_test = data_test[['from','to']].values
y_pred = [base_model.predict(x) for x in X_test]
# to pandas 
y_pred = pd.DataFrame(y_pred)

In [87]:
y_pred.to_csv('../data/flights_submission.csv',header=False,index=False)

In [88]:
y_pred

,0
0,11.588060
1,10.472639
2,11.507745
3,12.011550
4,11.138757
...,...
2227,10.582014
2228,10.713519
2229,10.278045
2230,11.295142
